# Reservoir operating policy - optimising the release policy
In this Notebook we will see how operating policy functions, together with simulation models and optimisation algorithms, can be used to assist the **long-term operation** of a water reservoir system. In order to avoid confusion, we define the operating policy as a function that can be used to determine the release conditional on the state of the reservoir system in the current time-step. In other words, rather than a release scheduling, the operating policy returns a strategy for making release decisions ([Dobson et al, 2019](https://doi.org/10.1016/j.advwatres.2019.04.012)).

<left><img src="Images/Dam3.gif" width = "500px"><left>

Once again we consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir **release policy** that given the historical inflows and water demand over the last years optimizes the **long-term** (several years) system performance.
<left> <img src="Images/system_representation_IO1.png" width = "600px"><left>

In long-term operation, reservoir release policies are often defined in the form of "**policy functions**" [(Loucks et al., 1981)](https://link.springer.com/book/10.1007/978-3-319-44234-1). A policy function generally defines a release value for a given value of storage. Usually higher storage values are associated to higher releases, e.g. for flood control purposes, and 
viceversa, e.g. if the storage is low, then less water is released to reduce the risk of future water shortages. Policy functions generally do not change from one year to the next but may vary depending on the time of year. Policy functions are typically derived from trial and error simulations that evaluate the impact of different policies on various reservoir system objectives.
<left><img src="Images/Policy_function.png" width = "400px"><left>

## Import libraries
To run this Notebook we need to import some necessary libraries. **Only if iRONs is run locally**: since one required library, [plotly](https://plot.ly/), is not available on Anaconda by default, you must have installed it first. Help on how to install libraries is given here: [How to install libraries](../0%20-%20Tutorials/0.b%20-%20How%20to%20install%20libraries.ipynb). If iRONs is run on the cloud, e.g. on [Binder](https://mybinder.org/) or [Microsoft Azure Notebooks](https://notebooks.azure.com/), we do not need to install the libraries to import them. 

Once all the necessary libraries are installed locally or in case we are running iRONs on the cloud, we import the libraries with the following code:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from platypus import NSGAII, Problem, Real, Integer # Import the optimizer

## Policy function
In this notebook we will generate a policy function for the regulated releases of the reservoir. Weekly release is scaled as a fraction of the long-term (five years or more) mean release (u_mean) while storage is scaled by the reservoir active capacity (0 at dead storage and 1 for full storage)

In [2]:
import sys
# Submodules
sys.path.append('../../Toolbox')
from Reservoir_operating_policy.Operating_policy_functions import four_points_policy

### 1.1 Parameters
Parameters of the policy function which are going to define the coordinates of the points delineating the function. u_min and u_max at minimum and maximum storage, u_mean is the long-term mean release and the coordinates u_ref and s_ref_1 and s_ref_2 of the (reference) inflection points. For example:

In [3]:
# System constraints
u_min = 2 # ML/week
u_max = 80 # ML/week
# Mean long term release
u_mean = 20 # ML/week
# Policy function parameters
u_ref = 20/u_mean
s_ref_1 = 0.2
s_ref_2 = 0.8

The policy function is delineated by 4 points (x0, x1, x2 and x3) which are defined by the parameters set in the cell above.

In [4]:
### Policy function constrains ###
u_0 = u_min/u_mean # release at storage fraction = 0
u_1 = u_max/u_mean # release at storage fraction = 1

### Policy function defining points ###
x0 = [0,       u_0]
x1 = [s_ref_1, u_ref]
x2 = [s_ref_2, u_ref]
x3 = [1,       u_1]

param = [x0, x1, x2, x3, u_mean]

Policy function

In [5]:
### Release fraction ###
u_frac = four_points_policy(param)/u_mean
### Storage fraction ###
s_frac = np.arange(0,1.01,0.01)

Plotting the reservoir policy function for regulated releases

In [6]:
# Axis characterisitcs
x_sc_0 = LinearScale(min=0,max=1.03);y_sc_0 = LinearScale(min=0,max=4)
x_ax_0 = Axis(label='Storage fraction', scale=x_sc_0)
y_ax_0 = Axis(label='Release fraction', scale=y_sc_0, orientation='vertical')
# Plot
policy_function_points = Scatter(x = [x0[0], x1[0], x2[0], x3[0]], 
                            y = [x0[1], x1[1], x2[1], x3[1]],
                            colors=['red'],stroke = 'lightgray',
                            scales={'x': x_sc_0, 'y': y_sc_0},
                            names = ['x0','x1','x2','x3'])
policy_function_0 = Lines(x = s_frac, y = u_frac,
                       colors=['blue'],stroke = 'lightgray',
                       scales={'x': x_sc_0, 'y': y_sc_0})
#Figure characteristics
fig_0 = plt.Figure(marks = [policy_function_0,policy_function_points],title = 'Policy function', axes=[x_ax_0, y_ax_0],
                    layout={'width': '500px', 'height': '450px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_0])

## Determining the release policy by trial and error (manual optimisation)
Here we want to use the reservoir model to assist the reservoir operator in determining the best release (u) policy evaluated against the historical inflows, evaporation and water demand data. 

### Historical inflows, evaporation and water demand data
Let's assume we want to look at the last 100 weeks (so ***N=100***). We generate the historical inflows, evaporation, regulated releases and demand data as arrays of 100 random numbers with a given mean (loc) and standard deviation (scale).

In [7]:
N = 100 # weeks
I_hist = np.abs(np.random.normal(loc=20,scale=10,size=N))
e_hist = np.abs(np.random.normal(loc=4,scale=1,size=N))
d_hist = np.abs(np.random.normal(loc=20,scale=5,size=N))
u_hist = np.abs(np.random.normal(loc=20,scale=1,size=N))

Plot the weekly inflow in the last 100 weeks:

In [8]:
# Axis characterisitcs
x_sc_1 = LinearScale();y_sc_1 = LinearScale(min=0,max=35)
x_ax_1 = Axis(label='week', scale=x_sc_1);y_ax_1 = Axis(label='ML/week', scale=y_sc_1, orientation='vertical')
# Bar plot
inflow_plot = plt.bar(np.arange(1,N+1),I_hist,colors=['blue'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['inflow'], display_legend = True)
#Figure characteristics
fig_1a = plt.Figure(marks = [inflow_plot],title = 'Inflow for the last 100 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1a])

Plot the weekly demand in the last 100 weeks:

In [9]:
# Bar plot (we use the same axis as the weekly inflows figure)
demand_plot   = plt.bar(np.arange(1,N+1),d_hist,colors=['gray'],stroke = 'lightgray',opacities = [0.7]*N, 
                        labels = ['demand'], display_legend = True, 
                    stroke_width = 1,scales={'x': x_sc_1, 'y': y_sc_1})
#Figure characteristics
fig_1b = plt.Figure(marks = [demand_plot],title = 'Demand for the last 100 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1b])

### Definition of other input parameters
Let's define other variables that are needed for the reservoir system simulation, such as the reservoir storage capacity, the environmental compensation flow, etc.

In [10]:
### Constraints ###
s_max = 150 #  (ML) Maximum storage (=reservoir capacity)
s_min = 0 # (ML) Minimum storage (set to zero for now)
u_max = 60 # (ML/week) Maximum release capacity
env_min = 2 # (ML/week)   # Environmental compensation flow

### Initial conditions ###
s_0 = 80 # (ML) # Storage volume at the beginning of the simulation period

### Policy function
The policy function is delineated by 4 points: x0, x1, x2 and x3.

In [11]:
u_mean = np.mean(u_hist)# mean historical release
u_0 = env_min/u_mean # release fraction at the minimum storage level (= environmental flow / mean release)
u_1 = u_max/u_mean # release at the maximum storage level (= max release capacity / mean release)

### Implementation of the reservoir simulation function
Here we define a function that implements the reservoir simulation, that is, iteratively apply the mass balance equation and reconstruct the temporal evolution of the reservoir variables over the simulation period

In [12]:
import sys
# Submodules
sys.path.append('../../Toolbox')
from Reservoir_system_simulation.Res_sys_sim import Res_sys_sim

### System objectives
Let's assume that, we are interested in minimising the deficit with respect to a historical water demand, that is, to minimise the objective function:

$$TSD = \sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-u(t) \ ) \ ]^2 $$

where N is the length of the simulation period that we are considering, and d(t) is the water demand for each time-interval in that period, and TSD stands for Total Squared Deficit. Notice that the function $max(0,...)$ enables us to only count the difference between demand d and release u when this is positive, that is, when the release u is smaller than the demand d, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. This translates the fact that small deficit amounts are easier to mitigate and hence more acceptable, while larger ones can cause disproportionately severe impacts and should be avoided as much as possible.

We are also interested in minimising the chances that the reservoir level go below a minimum threshold. This could be for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment. We measure how well this criterion is satisfied by the following objective function:

$$MSV = \sum_{t=1}^{N} [ \ max ( \ rc - s(t) , \ 0) \ ] $$

where, again, N is the length of the simulation period, s is the reservoir storage, and rc is the minimum reservoir storage threshold that should preferably not be transpassed (MSV stands for Minimum Storage Violation). 

For our case, let's set this threshold to 30 ML.

In [13]:
rc = np.array([40]*(N+1)) # (ML)  minimum reservoir storage threshold

### Determining the optimal release scheduling via interactive visualisation

Now use the sliders to modify the release policy delineated by the policy function, in a way that minimises the Minimum Storage Violation.

Function to update the release policy when changing the parameters with the sliders

In [14]:
def update_operating_policy_1(s_ref_1,s_ref_2,u_ref):
    if s_ref_1 > s_ref_2:
        s_ref_1 = s_ref_2   
    x0 = [0,       u_0]
    x1 = [s_ref_1, u_ref]
    x2 = [s_ref_2, u_ref]
    x3 = [1,       u_1]
    param = [x0, x1, x2, x3, u_mean]
    u_frac = four_points_policy(param)/u_mean
    
    Qreg = {'releases' : {'file_name' : 'Reservoir_operating_policy.Operating_policy_functions',
                         'function' : 'four_points_policy',
                         'param': param},
            'inflows' : [],
            'rel_inf' : []}
    
    Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(I_hist, e_hist, s_0, s_min, s_max, env_min, d_hist, Qreg)
    
    TSD = (np.sum((np.maximum(d_hist-u,[0]*N))**2)).astype('int')
    fig_1b.title = 'Supply vs Demand - TSD = '+str(TSD)+' ML^2'
    
    MSV = (np.sum((np.maximum(rc-s,[0]*(N+1))))).astype('int')
    fig_1c.title = 'Reservoir storage volume - MSV = '+str(MSV)+' ML'

    return u_frac, Qenv, Qspill, u, I_reg, s

Function to update the figures when changing the parameters with the sliders

In [15]:
def update_figure_1(change):
    policy_function.y = update_operating_policy_1(s_ref_1.value,s_ref_2.value,u_ref.value)[0]
    releases.y = update_operating_policy_1(s_ref_1.value,s_ref_2.value,u_ref.value)[3]
    storage.y = update_operating_policy_1(s_ref_1.value,s_ref_2.value,u_ref.value)[5]

Definition of the sliders

In [16]:
u_ref = widgets.FloatSlider(min=0.5, max=u_1, value=1, step=0.05,
                            description = 'u_ref: ',
                            continuous_update = False)
u_ref.observe(update_figure_1,names = 'value')

s_ref_1 = widgets.FloatSlider(min=0, max=1, value=0.25, step=0.05, 
                              description = 's_ref_1: ',
                              continuous_update=False)
s_ref_1.observe(update_figure_1,names = 'value')

s_ref_2 = widgets.FloatSlider(min=0, max=1, value=0.75, step=0.05,
                              description = 's_ref_2: ',
                              continuous_update=False)
s_ref_2.observe(update_figure_1,names = 'value')

Initial simulation applying the default slider values of the parameters 

In [18]:
x0 = [0,       u_0]
x1 = [s_ref_1.value, u_ref.value]
x2 = [s_ref_2.value, u_ref.value]
x3 = [1,       u_1]
param = [x0, x1, x2, x3, u_mean]
u_frac = four_points_policy(param)/u_mean

Qreg = {'releases' : {'file_name' : 'Reservoir_operating_policy.Operating_policy_functions',
                     'function' : 'four_points_policy',
                     'param': param},
        'inflows' : [],
        'rel_inf' : []}

Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(I_hist, e_hist, s_0, s_min, s_max, env_min, d_hist, Qreg)

Policy function

In [19]:
s_frac = np.arange(0,1.01,0.01)

x_sc_1a = LinearScale(min=0,max=1); y_sc_1a = LinearScale(min=0,max=u_1);
x_ax_1a = Axis(label='Storage fraction', scale=x_sc_1a); 
y_ax_1a = Axis(label='Release fraction', scale=y_sc_1a, orientation='vertical')

policy_function           = Lines(x   = s_frac,
                          y      = u_frac,
                          colors = ['blue'],
                          scales = {'x': x_sc_1a, 'y': y_sc_1a})

fig_1a             = plt.Figure(marks = [policy_function],
                               title = 'Policy function',
                               axes=[x_ax_1a, y_ax_1a],
                               layout={'width': '400px', 'height': '375px'}, 
                               animation_duration=1000,
                               scales={'x': x_sc_1a, 'y': y_sc_1a})

policy_function.observe(update_figure_1, ['x', 'y'])

Releases vs Demand

In [20]:
x_sc_1b = LinearScale(min=0,max=N);         y_sc_1b = LinearScale(min=0,max=u_max);
x_ax_1b = Axis(label='week', scale=x_sc_1b); y_ax_1b = Axis(label='ML/week', scale=y_sc_1b, orientation='vertical')

demand             = Bars(x   = np.arange(1,N+1),
                          y      = d_hist,
                          colors = ['gray'],
                          scales = {'x': x_sc_1b, 'y': y_sc_1b})

releases           = Bars(x   = np.arange(1,N+1),
                          y      = u,
                          colors = ['green'],
                          scales = {'x': x_sc_1b, 'y': y_sc_1b})

TSD = (np.sum((np.maximum(d_hist-u,[0]*N))**2)).astype('int')

fig_1b             = plt.Figure(marks = [demand, releases],
                               title = 'Supply vs Demand - TSD = '+str(TSD)+' ML^2',
                               axes=[x_ax_1b, y_ax_1b],
                               layout={'width': '950px', 'height': '250px'}, 
                               animation_duration=1000,
                               scales={'x': x_sc_1b, 'y': y_sc_1b})

releases.observe(update_figure_1, ['x', 'y'])

Storage

In [21]:
x_sc_1c = LinearScale();                    y_sc_1c = LinearScale(min=0,max=200);
x_ax_1c = Axis(label='week', scale=x_sc_1c); y_ax_1c = Axis(label='ML', scale=y_sc_1c, orientation='vertical')

storage           = Lines(x      = np.arange(0,N+1),
                          y      = s ,
                          colors = ['blue'],
                          scales = {'x': x_sc_1c, 'y': y_sc_1c},
                          fill   = 'bottom',fill_opacities = [0.8],fill_colors = ['blue'])

max_storage       = plt.plot(x=np.arange(0,N+1),
                             y=[s_max]*(N+1),
                             colors=['red'],
                             scales={'x': x_sc_1c, 'y': y_sc_1c})

max_storage_label = plt.label(text = ['Max storage'], 
                              x=[0],
                              y=[s_max+15],
                              colors=['red'])

min_storage = plt.plot(np.arange(0,N+1),rc,
                         scales={'x': x_sc_1c, 'y': y_sc_1c},
                         colors=['red'],opacities = [1],
                         line_style = 'dashed',
                         fill = 'bottom',fill_opacities = [0.4],fill_colors = ['red'], stroke_width = 1)
min_storage_label = plt.label(text = ['Min storage'], 
                                x=[0],
                                y=[rc[0]-10],
                                colors=['red'])

MSV = (np.sum((np.maximum(rc-s,[0]*(N+1))))).astype('int')

fig_1c             = plt.Figure(marks = [storage,max_storage,max_storage_label,
                                        min_storage,min_storage_label],
                               title = 'Reservoir storage volume - MSV = '+str(MSV)+' ML',
                               axes=[x_ax_1c, y_ax_1c],
                               layout={'width': '950px', 'height': '250px'}, 
                               animation_duration=1000,
                               scales={'x': x_sc_1c, 'y': y_sc_1c})

storage.observe(update_figure_1, ['x', 'y'])

Plot

In [22]:
Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [widgets.VBox([u_ref,s_ref_1,s_ref_2],layout=Box_layout), fig_1a],layout=Box_layout),fig_1b,fig_1c],layout=Box_layout)

## From manual to automatic optimization approach
As we have seen, when we deal with two conflicting objective, we cannot find a solution that optimise both simoultaneously. If we prioritize one objective, the other one is deteriorated: there is a trade-off between the two. It would then be interesting to explore this tradeoff, and find all the release schedules that produce a different optimal combination of the two objectives. However, this is too cumbersome to do manually. Here we then use a multi-objective optimisation algorithm to do that for us. 

To this end, we use the Python Platypus package, and the NSGAII algorithm implemented in it. For more information about these methods and tools, see [Deb et al, 2002](https://ieeexplore.ieee.org/document/996017) and the [Platypus webpage](https://platypus.readthedocs.io). The code to run the optimisation is the following:

In [24]:
def auto_optim(vars):

    u_ref   = vars[0]
    s_ref_1 = vars[1]
    s_ref_2 = vars[2]
    
    x0 = [0,       u_0]
    x1 = [s_ref_1, u_ref]
    x2 = [s_ref_2, u_ref]
    x3 = [1,       u_1]
    param = [x0, x1, x2, x3, u_mean]
    
    Qreg = {'releases' : {'file_name' : 'Reservoir_operating_policy.Operating_policy_functions',
                         'function' : 'four_points_policy',
                         'param': param},
            'inflows' : [],
            'rel_inf' : []}
    
    Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(I_hist, e_hist, s_0, s_min, s_max, env_min, d_hist, Qreg)
    
    TSD = (np.sum((np.maximum(d_hist-u,[0]*N))**2)).astype('int')
    MSV = (np.sum((np.maximum(rc-s,[0]*(N+1))))).astype('int')
    
    constraints = [s_ref_2-s_ref_1]
    
    return [TSD, MSV], constraints

problem = Problem(3,2,1)
Real0 = Real(0,u_1); Real1 = Real(0, 1); Real2 = Real(0, 1)

problem.types[:] = [Real0] + [Real1] + [Real2]
problem.constraints[:] = ">=0"
problem.function = auto_optim

population_size = 30
algorithm = NSGAII(problem,population_size)
algorithm.run(1000) # Number of iterations

results1_optim = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

sol_optim = [algorithm.result[i].variables for i in range(population_size)]

Function to update the release policy when changing the parameters with the sliders

In [25]:
def update_operating_policy_2(i):
    
    u_ref,s_ref_1,s_ref_2 = sol_optim[i]
    x0 = [0,       u_0]
    x1 = [s_ref_1, u_ref]
    x2 = [s_ref_2, u_ref]
    x3 = [1,       u_1]
    param = [x0, x1, x2, x3, u_mean]
    u_frac = four_points_policy(param)/u_mean
    
    Qreg = {'releases' : {'file_name' : 'Reservoir_operating_policy.Operating_policy_functions',
                         'function' : 'four_points_policy',
                         'param': param},
            'inflows' : [],
            'rel_inf' : []}
    
    Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(I_hist, e_hist, s_0, s_min, s_max, env_min, d_hist, Qreg)
    
    MSV = (np.sum((np.maximum(rc-s,[0]*(N+1))))).astype('int')
    fig_2c.title = 'Reservoir storage volume - MSV = '+str(MSV)+' ML'
    
    TSD = (np.sum((np.maximum(d_hist-u,[0]*N))**2)).astype('int')
    fig_2b.title = 'Supply vs Demand - Total squared deficit = '+str(TSD)+' ML^2'
    
    return u_frac, Qenv, Qspill, u, I_reg, s

Function to update the figure when changing the parameters with the sliders

In [26]:
def update_figure_2(change):
    
    policy_function.y = update_operating_policy_2(pareto_front.selected[0])[0]
    releases.y = update_operating_policy_2(pareto_front.selected[0])[3]
    storage.y = update_operating_policy_2(pareto_front.selected[0])[5]

Pareto front

In [27]:
x_sc_pf = LinearScale();y_sc_pf = LinearScale()
x_ax_pf = Axis(label='Total squared deficit [ML^2]', scale=x_sc_pf)
y_ax_pf = Axis(label='Minimum storage violation [ML]', scale=y_sc_pf, orientation='vertical')

pareto_front = plt.scatter(results1_optim[:],results2_optim[:],
                           scales={'x': x_sc_pf, 'y': y_sc_pf},
                           colors=['deepskyblue'], 
                           interactions={'hover':'tooltip','click': 'select'})

pareto_front.unselected_style={'opacity': 0.4}
pareto_front.selected_style={'fill': 'red', 'stroke': 'yellow', 'width': '1125px', 'height': '125px'}
def_tt = Tooltip(fields=['index','x', 'y'],
                 labels=['index','Water deficit', 'Min storage'], 
                 formats=['.d','.1f', '.1f'])
pareto_front.tooltip=def_tt

fig_pf = plt.Figure(marks = [pareto_front],title = 'Interactive Pareto front', 
                    axes=[x_ax_pf, y_ax_pf],
                    layout={'width': '400px', 'height': '400px'}, animation_duration=1000)

if pareto_front.selected == []:
    pareto_front.selected = [0]

pareto_front.observe(update_figure_2,'selected')

Initial simulation applting the point of the Pareto Fron selected by default

In [29]:
u_ref,s_ref_1,s_ref_2 = sol_optim[pareto_front.selected[0]]
x0 = [0,       u_0]
x1 = [s_ref_1, u_ref]
x2 = [s_ref_2, u_ref]
x3 = [1,       u_1]
param = [x0, x1, x2, x3, u_mean]
u_frac = four_points_policy(param)/u_mean

Qreg = {'releases' : {'file_name' : 'Reservoir_operating_policy.Operating_policy_functions',
                     'function' : 'four_points_policy',
                     'param': param},
        'inflows' : [],
        'rel_inf' : []}

Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(I_hist, e_hist, s_0, s_min, s_max, env_min, d_hist, Qreg)

Policy function figure

In [30]:
s_frac = np.arange(0,1.01,0.01)

x_sc_2a = LinearScale(min=0,max=1); y_sc_2a = LinearScale(min=0,max=u_max/u_mean);
x_ax_2a = Axis(label='Storage fraction', scale=x_sc_2a); 
y_ax_2a = Axis(label='Release fraction', scale=y_sc_2a, orientation='vertical')

policy_function           = Lines(x   = s_frac,
                          y      = u_frac ,
                          colors = ['blue'],
                          scales = {'x': x_sc_2a, 'y': y_sc_2a})

fig_2a             = plt.Figure(marks = [policy_function],
                               title = 'Policy function',
                               axes=[x_ax_2a, y_ax_2a],
                               layout={'width': '400px', 'height': '375px'}, 
                               animation_duration=1000,
                               scales={'x': x_sc_2a, 'y': y_sc_2a})

policy_function.observe(update_figure_2, ['x', 'y'])

Releases vs Demand figure

In [31]:
x_sc_2b = LinearScale(min=0,max=N);         y_sc_2b = LinearScale(min=0,max=u_max);
x_ax_2b = Axis(label='week', scale=x_sc_2b); y_ax_2b = Axis(label='ML/week', scale=y_sc_2b, orientation='vertical')

demand             = Bars(x   = np.arange(1,N+1),
                          y      = d_hist,
                          colors = ['gray'],
                          scales = {'x': x_sc_2b, 'y': y_sc_2b})

releases           = Bars(x   = np.arange(1,N+1),
                          y      = u,
                          colors = ['green'],
                          scales = {'x': x_sc_2b, 'y': y_sc_2b})

TSD = (np.sum((np.maximum(d_hist-u,[0]*N))**2)).astype('int')

fig_2b             = plt.Figure(marks = [demand, releases],
                               title = 'Supply vs Demand - TSD = '+str(TSD)+' ML^2',
                               axes=[x_ax_2b, y_ax_2b],
                               layout={'width': '950px', 'height': '250px'}, 
                               animation_duration=1000,
                               scales={'x': x_sc_2b, 'y': y_sc_2b})

releases.observe(update_figure_2, ['x', 'y'])

Storage figure

In [32]:
x_sc_2c = LinearScale();                    y_sc_2c = LinearScale(min=0,max=200);
x_ax_2c = Axis(label='week', scale=x_sc_2c); y_ax_2c = Axis(label='ML', scale=y_sc_2c, orientation='vertical')

storage           = Lines(x      = np.arange(0,N+1),
                          y      = s ,
                          colors = ['blue'],
                          scales = {'x': x_sc_2c, 'y': y_sc_2c},
                          fill   = 'bottom',fill_opacities = [0.8],fill_colors = ['blue'])

max_storage       = plt.plot(x=np.arange(0,N+1),
                             y=[s_max]*(N+1),
                             colors=['red'],
                             scales={'x': x_sc_2c, 'y': y_sc_2c})

max_storage_label = plt.label(text = ['Max storage'], 
                              x=[0],
                              y=[s_max+15],
                              colors=['red'])

min_storage = plt.plot(np.arange(0,N+1),rc,
                         scales={'x': x_sc_2c, 'y': y_sc_2c},
                         colors=['red'],opacities = [1],
                         line_style = 'dashed',
                         fill = 'bottom',fill_opacities = [0.4],fill_colors = ['red'], stroke_width = 1)
min_storage_label = plt.label(text = ['Min storage'], 
                                x=[0],
                                y=[rc[0]-10],
                                colors=['red'])

MSV = (np.sum((np.maximum(rc-s,[0]*(N+1))))).astype('int')

fig_2c             = plt.Figure(marks = [storage,max_storage,max_storage_label,
                                        min_storage,min_storage_label],
                               title = 'Reservoir storage volume - MSV = '+str(MSV)+' ML',
                               axes=[x_ax_2c, y_ax_2c],
                               layout={'width': '950px', 'height': '250px'}, 
                               animation_duration=1000,
                               scales={'x': x_sc_2c, 'y': y_sc_2c})

storage.observe(update_figure_2, ['x', 'y'])

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot, called Pareto front, which displays the combination of the two objective values in correspondence to a set of optimised solutions. Click on one point in the Pareto front to visualise the release scheduling that generates that performance, and associated storage time series.  What do you think would be a balanced solution?

In [33]:
Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [fig_pf, fig_2a],layout=Box_layout),fig_2b,fig_2c],layout=Box_layout)

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

Dobson B. et al (2019) An argument-driven classification and comparison of reservoir operation optimization methods, Advances in Water Resources, 128, 74-86.

Loucks D. P. et al (1981) Water resource systems planning and analysis, Prentice-Hall.

# Questionnaire: Section 2 of 2
Now that you are done with the Notebooks, could you please answer the questions of Section 2 of 2 of the questionnaire (click on Next button that you will find at the end of Section 1 of 2 of the questionnaire)?